# 5. LangChain Expression Language（LCEL）徹底解説


In [9]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [10]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 langchain-community==0.3.0

## 5.1. Runnable と RunnableSequence―LCEL の最も基本的な構成要素


In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [12]:
prompt_value = prompt.invoke({"dish": "カレー"})
ai_message = model.invoke(prompt_value)
output = output_parser.invoke(ai_message)

print(output)

カレーのレシピをご紹介します！以下は基本的なチキンカレーのレシピです。

### 材料（4人分）
- 鶏もも肉：400g（食べやすい大きさにカット）
- 玉ねぎ：2個（みじん切り）
- にんにく：2片（みじん切り）
- 生姜：1片（みじん切り）
- トマト：1個（ざく切り）
- カレーパウダー：大さじ2
- クミンシード：小さじ1（お好みで）
- ココナッツミルク：200ml（お好みで）
- サラダ油：大さじ2
- 塩：適量
- 黒胡椒：適量
- 水：400ml
- パクチー（飾り用）：適量

### 作り方
1. **下準備**: 鶏肉に塩と黒胡椒を振りかけて下味をつけておきます。

2. **玉ねぎを炒める**: 大きめの鍋にサラダ油を熱し、みじん切りにした玉ねぎを加え、中火で透明になるまで炒めます。

3. **香味野菜を加える**: にんにくと生姜を加え、香りが立つまでさらに炒めます。

4. **スパイスを加える**: クミンシードを加え、さらに1分ほど炒めた後、カレーパウダーを加え、全体がよく混ざるまで炒めます。

5. **鶏肉を加える**: 鶏肉を鍋に加え、表面が白くなるまで炒めます。

6. **トマトと水を加える**: ざく切りにしたトマトと水を加え、全体をよく混ぜます。沸騰したら、弱火にして蓋をし、約20分煮込みます。

7. **ココナッツミルクを加える**: 煮込みが終わったら、ココナッツミルクを加え、さらに5分ほど煮込みます。味を見て、必要に応じて塩で調整します。

8. **盛り付け**: お皿に盛り付け、パクチーを散らして完成です。

### 提供方法
ご飯やナンと一緒にお楽しみください。お好みでヨーグルトやサラダを添えると、より美味しくいただけます。

ぜひお試しください！


In [14]:
chain = prompt | model | output_parser

In [15]:
output = chain.invoke({"dish": "カレー"})
print(output)

カレーのレシピをご紹介します。シンプルで美味しい基本のカレーを作りましょう。

### 材料（4人分）
- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約200g）
- サラダ油: 大さじ2
- 水: 800ml
- 塩: 適量
- 胡椒: 適量
- お好みでガーリックパウダーや生姜: 適量

### 作り方
1. **材料の下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒を振っておきます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**:
   - 水を加え、強火で煮立たせます。煮立ったら火を弱め、アクを取りながら約15分煮ます。

5. **カレールーを加える**:
   - カレールーを割り入れ、よく溶かします。さらに10分ほど煮込み、全体がなじんだら味を見て、必要に応じて塩や胡椒で調整します。

6. **仕上げ**:
   - お好みでガーリックパウダーや生姜を加えて風味をアップさせます。火を止めて、少し冷ますと味がなじみます。

7. **盛り付け**:
   - ご飯と一緒に盛り付けて、お好みで福神漬けやらっきょうを添えて完成です。

### おすすめのトッピング
- 煮卵
- チーズ
- パクチー

この基本のカレーはアレンジがしやすいので、野菜や肉を変えて楽しんでください！


### Runnable の実行方法―invoke・stream・batch


In [16]:
chain = prompt | model | output_parser

for chunk in chain.stream({"dish": "カレー"}):
    print(chunk, end="", flush=True)

カレーのレシピをご紹介します。シンプルで美味しい基本のカレーを作りましょう。

### 材料（4人分）
- 鶏肉（もも肉または胸肉）: 400g
- 玉ねぎ: 2個
- にんじん: 1本
- じゃがいも: 2個
- カレールー: 1箱（約200g）
- サラダ油: 大さじ2
- 水: 800ml
- 塩: 適量
- 胡椒: 適量
- お好みでガーリックパウダーや生姜: 適量

### 作り方
1. **材料の下ごしらえ**:
   - 鶏肉は一口大に切り、塩と胡椒をふっておきます。
   - 玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切ります。

2. **炒める**:
   - 大きめの鍋にサラダ油を熱し、玉ねぎを中火で炒めます。玉ねぎが透明になるまで炒めます。
   - 鶏肉を加え、表面が白くなるまで炒めます。

3. **野菜を加える**:
   - にんじんとじゃがいもを鍋に加え、全体をよく混ぜます。

4. **煮る**:
   - 水を加え、強火で煮立たせます。煮立ったら、アクを取り除き、中火にして蓋をし、約15分煮ます。

5. **カレールーを加える**:
   - 火を止めてカレールーを加え、よく溶かします。再び弱火にし、10分ほど煮込みます。お好みでガーリックパウダーや生姜を加えて風味を調整します。

6. **仕上げ**:
   - 味を見て、必要であれば塩で調整します。全体がなじんだら、火を止めます。

7. **盛り付け**:
   - ご飯と一緒に盛り付けて、お好みで福神漬けやらっきょうを添えて完成です。

### おすすめのトッピング
- 煮卵
- チーズ
- 青ねぎ

この基本のカレーはアレンジがしやすいので、野菜や肉を変えて楽しんでください！

In [17]:
chain = prompt | model | output_parser

outputs = chain.batch([{"dish": "カレー"}, {"dish": "うどん"}])
print(outputs)

['カレーのレシピをご紹介します！以下は基本的なチキンカレーのレシピです。\n\n### 材料（4人分）\n- 鶏もも肉：400g（食べやすい大きさにカット）\n- 玉ねぎ：2個（みじん切り）\n- にんにく：2片（みじん切り）\n- 生姜：1片（みじん切り）\n- トマト：1個（ざく切り）\n- カレーパウダー：大さじ2\n- クミンパウダー：小さじ1\n- ココナッツミルク：200ml（お好みで）\n- サラダ油：大さじ2\n- 塩：適量\n- 黒胡椒：適量\n- 水：400ml\n- パクチー（飾り用）：適量\n\n### 作り方\n1. **下ごしらえ**: 鶏肉に塩と黒胡椒をふり、下味をつけておきます。\n2. **玉ねぎを炒める**: 大きめの鍋にサラダ油を熱し、みじん切りにした玉ねぎを加え、透明になるまで中火で炒めます。\n3. **香味野菜を加える**: にんにくと生姜を加え、香りが立つまでさらに炒めます。\n4. **鶏肉を加える**: 鶏肉を鍋に加え、表面が白くなるまで炒めます。\n5. **スパイスを加える**: カレーパウダーとクミンパウダーを加え、全体に絡めるように炒めます。\n6. **トマトと水を加える**: ざく切りにしたトマトと水を加え、煮立たせます。アクが出たら取り除きます。\n7. **煮込む**: 蓋をして中弱火で約20分煮込みます。鶏肉が柔らかくなったら、ココナッツミルクを加え、さらに5分煮ます。\n8. **味を調える**: 塩で味を調整し、必要に応じてスパイスを追加します。\n9. **盛り付け**: お皿に盛り付け、パクチーを散らして完成です。\n\n### 提供方法\nご飯やナンと一緒にお楽しみください。お好みでヨーグルトやサラダを添えると、より美味しくいただけます。\n\nぜひお試しください！', 'うどんのレシピをご紹介します。シンプルで美味しい「かけうどん」の作り方です。\n\n### 材料（2人分）\n- うどん（乾燥または生）: 2玉\n- だし汁: 600ml\n  - だしの素: 大さじ1（または昆布と鰹節を使って自家製だしを取る）\n  - 水: 600ml\n- 醤油: 大さじ2\n- みりん: 大さじ1\n- 塩: 少々\n- トッピング（お好みで）:\n  - ネギ（小口切り）\n  - 天かす

### LCEL の「|」で様々な Runnable を連鎖させる


In [18]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [19]:
cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーの質問にステップバイステップで回答してください。"),
        ("human", "{question}"),
    ]
)

cot_chain = cot_prompt | model | output_parser

In [20]:
summarize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ステップバイステップで考えた回答から結論だけ抽出してください。"),
        ("human", "{text}"),
    ]
)

summarize_chain = summarize_prompt | model | output_parser

In [25]:
cot_summarize_chain = cot_chain | summarize_chain
output = cot_summarize_chain.invoke({"question": "10 + 2 * 3"})
print(output)

10 + 2 * 3 の答えは **16** です。


## 5.2. RunnableLambda―任意の関数を Runnable にする


In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [27]:
from langchain_core.runnables import RunnableLambda


def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | RunnableLambda(upper)

ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### chain デコレーターを使った RunnableLambda の実装


In [28]:
from langchain_core.runnables import chain


@chain
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | upper

ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### RunnableLambda への自動変換


In [30]:
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | upper

In [31]:
ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### Runnable の入力の型と出力の型に注意


In [34]:
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | upper

# 以下のコードを実行するとエラーになります
# output = chain.invoke({"input": "Hello!"})

In [36]:
chain = prompt | model | StrOutputParser() | upper

In [37]:
output = chain.invoke({"input": "Hello!"})
print(output)

HELLO! HOW CAN I ASSIST YOU TODAY?


### （コラム）独自の関数を stream に対応させたい場合


In [39]:
from typing import Iterator


def upper(input_stream: Iterator[str]) -> Iterator[str]:
    for text in input_stream:
        yield text.upper()


chain = prompt | model | StrOutputParser() | upper

for chunk in chain.stream({"input": "Hello!"}):
    print(chunk, end="", flush=True)

HELLO! HOW CAN I ASSIST YOU TODAY?

## 5.3. RunnableParallel―複数の Runnable を並列で処理する


In [40]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

In [41]:
optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは楽観主義者です。ユーザーの入力に対して楽観的な意見をください。"),
        ("human", "{topic}"),
    ]
)
optimistic_chain = optimistic_prompt | model | output_parser

In [42]:
pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは悲観主義者です。ユーザーの入力に対して悲観的な意見をください。"),
        ("human", "{topic}"),
    ]
)
pessimistic_chain = pessimistic_prompt | model | output_parser

In [43]:
import pprint
from langchain_core.runnables import RunnableParallel

parallel_chain = RunnableParallel(
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
)

output = parallel_chain.invoke({"topic": "生成AIの進化について"})
pprint.pprint(output)

{'optimistic_opinion': '生成AIの進化は本当に素晴らしいですね！技術が進むことで、私たちの生活がより便利で豊かになる可能性が広がっています。クリエイティブな作業や問題解決の手助けをしてくれるAIが増えてきて、私たちのアイデアを実現するためのパートナーとして活躍しています。これからも新しい発見や革新が続くことで、私たちの未来はますます明るくなるでしょう！どんな新しい可能性が待っているのか、ワクワクしますね。',
 'pessimistic_opinion': '生成AIの進化は確かに目覚ましいものがありますが、その一方で多くの懸念も伴っています。技術が進化することで、私たちの仕事が奪われたり、情報の信頼性が低下したりするリスクが高まっています。さらに、AIが生成するコンテンツが増えることで、オリジナリティやクリエイティビティが失われる可能性もあります。結局のところ、便利さの裏には多くの問題が潜んでいるのではないでしょうか。私たちがこの技術をどのように扱うかが、未来に大きな影響を与えることは間違いありませんが、その未来が明るいとは限らないのが現実です。'}


### RunnableParallel の出力を Runnable の入力に連結する


In [44]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "あなたは客観的AIです。2つの意見をまとめてください。"),
        ("human", "楽観的意見: {optimistic_opinion}\n悲観的意見: {pessimistic_opinion}"),
    ]
)

In [45]:
synthesize_chain = (
    RunnableParallel(
        {
            "optimistic_opinion": optimistic_chain,
            "pessimistic_opinion": pessimistic_chain,
        }
    )
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化については、楽観的な意見と悲観的な意見が存在します。楽観的な見方では、生成AIの技術が進化することで、私たちの生活が便利で豊かになる可能性が広がり、クリエイティブな作業や問題解決のパートナーとしての役割を果たすことが期待されています。この進化により、新しい発見や革新が続き、未来が明るくなることへの期待感が高まっています。

一方で、悲観的な見方では、生成AIの進化には多くの懸念が伴い、仕事の喪失や情報の信頼性の低下といったリスクが増大することが指摘されています。また、AIが生成するコンテンツが人間の創造性を脅かし、思考や感情に悪影響を及ぼす可能性も懸念されています。このように、便利さの裏には不安がつきまとい、未来が不透明になるのではないかという懸念も存在します。

総じて、生成AIの進化は多くの可能性を秘めている一方で、リスクや懸念も無視できないという複雑な状況が浮かび上がります。


### RunnableParallel への自動変換


In [46]:
synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
    | synthesize_prompt
    | model
    | output_parser
)

In [47]:
output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化については、楽観的な意見と悲観的な意見が存在します。楽観的な意見では、生成AIが私たちの生活を便利で豊かにし、クリエイティブな作業や問題解決のパートナーとしての役割を果たすことが期待されています。また、教育や医療、エンターテインメントなどの分野での革新を促進し、より多くの人々が新しい知識や体験にアクセスできるようになることが、社会全体の進化につながるとされています。

一方で、悲観的な意見では、生成AIの進化が仕事の喪失や情報の信頼性の低下を引き起こすリスクがあると指摘されています。AIが生成するコンテンツの氾濫により、真実と虚偽の区別が難しくなり、社会が混乱する可能性も懸念されています。このように、技術の進化は必ずしも良い結果をもたらすわけではなく、新たな問題を引き起こす要因となるかもしれません。

総じて、生成AIの進化には多くの可能性と同時にリスクも伴うため、慎重な対応が求められています。


### RunnableLambda との組み合わせ―itemgetter を使う例


In [48]:
from operator import itemgetter

topic_getter = itemgetter("topic")
topic = topic_getter({"topic": "生成AIの進化について"})
print(topic)

生成AIの進化について


In [50]:
from operator import itemgetter

synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "あなたは客観的AIです。{topic}について2つの意見をまとめてください。",
        ),
        (
            "human",
            "楽観的意見: {optimistic_opinion}\n悲観的意見: {pessimistic_opinion}",
        ),
    ]
)

synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
        "topic": itemgetter("topic"),
    }
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "生成AIの進化について"})
print(output)

生成AIの進化に関する意見は、楽観的な見方と悲観的な見方の2つに分かれます。

**楽観的意見:** 生成AIの進化は、私たちの生活をより便利で豊かにする可能性を秘めています。新しい技術が登場することで、クリエイティブな作業や問題解決の支援が行われ、私たちのアイデアや想像力を引き出す手助けをしてくれるでしょう。未来において、生成AIがどのように社会を変革していくのか、期待と興奮を持って見守ることができます。

**悲観的意見:** 一方で、生成AIの進化には多くの懸念も伴います。技術の進化が進むことで、仕事の喪失や情報の信頼性の低下といったリスクが増大します。また、AIが生成するコンテンツが氾濫することで、真実と虚偽の区別が難しくなり、社会全体が混乱する可能性もあります。このように、進化が必ずしも良い結果をもたらすわけではないという視点も重要です。


## 5.4. RunnablePassthrough―入力をそのまま出力する


In [52]:
import os
from google.colab import userdata

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

SecretNotFoundError: Secret TAVILY_API_KEY does not exist.

In [ ]:
!pip install tavily-python==0.5.0

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke("東京の今日の天気は？")
print(output)

### assign―RunnableParallel に値を追加する


In [ ]:
import pprint

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | RunnablePassthrough.assign(answer=prompt | model | StrOutputParser())

output = chain.invoke("東京の今日の天気は？")
pprint.pprint(output)

In [ ]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

In [ ]:
output = chain.invoke("東京の今日の天気は？")
print(output)

#### ＜補足：pick ＞


In [ ]:
chain = (
    RunnableParallel(
        {
            "question": RunnablePassthrough(),
            "context": retriever,
        }
    )
    .assign(answer=prompt | model | StrOutputParser())
    .pick(["context", "answer"])
)

In [ ]:
output = chain.invoke("東京の今日の天気は？")
print(output)

### （コラム）astream_events


In [ ]:
# Google Colabでは次のコードの「async」の箇所に「Use of "async" not allowed outside of async function」と表示されますが、エラーなく実行できます

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

async for event in chain.astream_events("東京の今日の天気は？", version="v2"):
    print(event, flush=True)

In [ ]:
async for event in chain.astream_events("東京の今日の天気は？", version="v2"):
    event_kind = event["event"]

    if event_kind == "on_retriever_end":
        print("=== 検索結果 ===")
        documents = event["data"]["output"]
        for document in documents:
            print(document)

    elif event_kind == "on_parser_start":
        print("=== 最終出力 ===")

    elif event_kind == "on_parser_stream":
        chunk = event["data"]["chunk"]
        print(chunk, end="", flush=True)

### （コラム）Chat history と Memory


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

chain = prompt | model | StrOutputParser()

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def respond(session_id: str, human_message: str) -> str:
    chat_message_history = SQLChatMessageHistory(
        session_id=session_id, connection="sqlite:///sqlite.db"
    )

    ai_message = chain.invoke(
        {
            "chat_history": chat_message_history.get_messages(),
            "input": human_message,
        }
    )

    chat_message_history.add_user_message(human_message)
    chat_message_history.add_ai_message(ai_message)

    return ai_message

In [ ]:
from uuid import uuid4

session_id = uuid4().hex

output1 = respond(
    session_id=session_id,
    human_message="こんにちは！私はジョンと言います！",
)
print(output1)

output2 = respond(
    session_id=session_id,
    human_message="私の名前が分かりますか？",
)
print(output2)